<a href="https://colab.research.google.com/github/edwinibague/Proyecto_De_Grado/blob/master/Caracteristicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#extracción de caracteristicas de imagenes de dataset

##Toma de dataset y alamacenamiento en nube


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#agregar bien el path 
!ls /content/drive/My\ Drive/dataset/
!cp -r '/content/drive/My Drive/dataset' /usr/local
!ls /usr/local/dataset

benignas_P.rar	benignasx.rar  Malignas.csv  malignas_P.rar  Malignas.rar
benignas_P.rar	benignasx.rar  Malignas.csv  malignas_P.rar  Malignas.rar


In [ ]:
!unrar e /usr/local/dataset/Malignas.rar -d /usr/local/dataset/Malignas/
!unrar e /usr/local/dataset/benignasx.rar -d /usr/local/dataset/benignasx/
!unrar e /usr/local/dataset/malignas_P.rar -d /usr/local/dataset/malignas_P/
!unrar e /usr/local/dataset/benignas_P.rar -d /usr/local/dataset/benignas_P/
!ls /usr/local/dataset/


Se han truncado las últimas 5000 líneas del flujo de salida.
Extracting  /usr/local/dataset/benignasx/ISIC_3206957.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3208873.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3209641.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3211246.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3212050.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3213469.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3215316.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3216049.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3216095.jpg                 31%  OK 
Extracting  /usr/local/dataset/benignasx/ISIC_3216482.jpg                 31%  OK 
Extracting  /usr/lo

In [ ]:
!rmdir /usr/local/dataset/Malignas.rar
!rmdir /usr/local/dataset/benignasx.rar
!rmdir /usr/local/dataset/malignas_P.rar
!rmdir /usr/local/dataset/benignas_P.rar

rmdir: failed to remove '/usr/local/dataset/Malignas.rar': Not a directory
rmdir: failed to remove '/usr/local/dataset/benignasx.rar': Not a directory
rmdir: failed to remove '/usr/local/dataset/malignas_P.rar': Not a directory
rmdir: failed to remove '/usr/local/dataset/benignas_P.rar': Not a directory



##procesamiento de dataset

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import json
import pandas as pd
from pandas import ExcelWriter
from PIL import ImageFile
from os import scandir, getcwd
from google.colab.patches import cv2_imshow


###Lectura Datos isic archivo .csv 

In [ ]:
df = pd.read_csv('/usr/local/dataset/Malignas.csv')
df

,_id,name,meta.clinical.age_approx,meta.clinical.anatom_site_general,meta.clinical.benign_malignant,meta.clinical.clin_size_long_diam_mm,meta.clinical.diagnosis,meta.clinical.diagnosis_confirm_type,meta.clinical.family_hx_mm,meta.clinical.lesion_id,meta.clinical.mel_class,meta.clinical.mel_mitotic_index,meta.clinical.mel_thick_mm,meta.clinical.mel_type,meta.clinical.mel_ulcer,meta.clinical.melanocytic,meta.clinical.nevus_type,meta.clinical.patient_id,meta.clinical.personal_hx_mm,meta.clinical.sex,meta.acquisition.acquisition_day,meta.acquisition.blurry,meta.acquisition.color_tint,meta.acquisition.dermoscopic_type,meta.acquisition.hairy,meta.acquisition.image_type,meta.acquisition.marker_pen,meta.acquisition.pixelsX,meta.acquisition.pixelsY
0,5436e3acbae478396759f0d3,ISIC_0000002,60.0,upper extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1022,767
1,5436e3acbae478396759f0d7,ISIC_0000004,80.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1022,767
2,5436e3adbae478396759f0e9,ISIC_0000013,30.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1022,767
3,5436e3aebae478396759f0fb,ISIC_0000022,55.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1504,1129
4,5436e3aebae478396759f103,ISIC_0000026,30.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,1504,1129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,5aaf162511659769136470dd,ISIC_0034294,70.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,600,450
2281,5aaf162711659769136471cf,ISIC_0034313,55.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,600,450
2282,5aaf162711659769136471f0,ISIC_0034316,85.0,upper extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,male,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,600,450
2283,5aaf162711659769136471fb,ISIC_0034317,70.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,female,NaN,NaN,NaN,NaN,NaN,dermoscopic,NaN,600,450


In [ ]:
def dataisic(df):
    age = df['meta.clinical.age_approx']
    sex = df['meta.clinical.sex']
    Sex = []
    Sex_prom = []
    for i in range(len(age)):
        if sex[i] == 'male':
            Sex_prom.append(1)
        elif sex[i] == 'female':
            Sex_prom.append(0)

    SexP = np.mean(Sex_prom)
    if SexP >= 0.5:
      SexP = 1
    else:
      SexP = 0

    for i in range(len(age)):   
        if sex[i] == 'male':
            Sex.append(1)
        elif sex[i] == 'female':
            Sex.append(0)
        else:
          Sex.append(SexP)

    Age_prom = np.mean(age)
    Age =[]
    for i in range(len(age)):
        if np.isnan(age[i]):
            Age.append(Age_prom)
        else:
            Age.append(age[i])

    return Age, Sex

In [ ]:
Age, Sex = dataisic(df)
len(Sex)
#len(Age)

2285

In [ ]:
def saveas(name):
    namesearch = df['name']
    pos =  0
    for i in range(len(namesearch)):
        if namesearch[i] == name:
            pos  =  i 
            break

    return pos

In [ ]:
nombre= 'ISIC_0033886'
ver = saveas(nombre)
ver

2181

###Lectura imagenes 

In [ ]:
RutMalignas = '/usr/local/dataset/Malignas'
RutBenignas = '/usr/local/dataset/benignasx'

In [ ]:
def Ruta(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [ ]:
NameMalig = Ruta(RutMalignas)
NameBenig = Ruta(RutBenignas)
ImgMalig = []

In [ ]:
def MaxCont(contour):
    Max = 0
    pos = 0
    
    for i in range(len(contour)):
        Dato = len(contour[i])
        if Dato > Max:
          Max = Dato
          pos = i
          
        else:
          Max = Max
          
    return pos

In [ ]:
def Borde(posicion, contorno, cx, cy):
    Magnitud = []
    index = 0
    a = 0
    b = 0
    prom = 0

    for i in range(360):
        Mag = (np.sqrt([np.power((contorno[posicion][index][0][0] - cx), 2) + np.power((contorno[posicion][index][0][1] -
                                                                                       cy), 2)]))[0]
        Magnitud.append(Mag)
        
        index += int((len(contorno[posicion]))/360)
        a += 1
        if a == 2:
            index += 1
            a = 0
        b += 1
        if b == 5:
            index += 1
            b = 0
        if index >= len(contorno[posicion]):
            index = len(contorno[posicion]) - 1

    for i in range(len(Magnitud)-1):
        prom += Magnitud[i]
    
    prom = prom/len(Magnitud)
    for i in range(len(Magnitud)):
          Magnitud[i] = Magnitud[i] - prom
    
    return Magnitud


In [ ]:
def Histograma(Imagen):
    color = ('b','g','r')
    #for i , c in enumerate(color):
        
    hist = cv2.calcHist([Imagen], [0], None, [256], [0, 256])
    hist[0][0] = 0
    
    imgB = cv2.equalizeHist(Imagen[:,0])
    imgG = cv2.equalizeHist(Imagen[:,1])
    imgR = cv2.equalizeHist(Imagen[:,2])

    return hist, imgB, imgG, imgR

Obtencion de Descriptores

In [ ]:
def Descriptors(RutaDataset, NameDataset):

    MagDat = []
    FourDat = []
    total = 0
    HistB = []
    HistG = []
    HistR = []

    for i in range(len(NameDataset)):
        cadena = RutaDataset + '/' + NameDataset[i]
        #print(NameDataset[i])
    

        img = cv2.imread(cadena)
        try:
          img = cv2.resize(img, (962, 722), interpolation = cv2.INTER_AREA)
        except:
          continue
        imgc = img.copy()
    
        #escala de grices y RGB
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        #cv2_imshow(gray)
        #threshold
        ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
        #contornos
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
          continue
        Maxcountor = MaxCont(contours)
        
        #centroide

        cnt = contours[Maxcountor]
        M = cv2.moments(cnt)
        #centro
        cx = int(M['m10']/(M['m00']+1e-5))
        cy = int(M['m01']/(M['m00']+ 1e-5))
        cv2.circle(thresh, (cx, cy), 1, (0,0,255), -1)

        Mag = Borde(Maxcountor, contours, cx, cy)

        #fourier de magnitud
        FourMag = np.fft.fft(Mag)

        #Histograma
        imagen =cv2.bitwise_and(imgc,imgc,mask=thresh)
        #Hist, ImgB, ImgG, ImgR = Histograma(imagen)

        imagen_rgb = cv2.bitwise_and(rgb,rgb,mask=thresh)
        imagen_hsv = cv2.cvtColor(imagen_rgb, cv2.COLOR_RGB2HSV)
        #componente_h = imagen_hsv[:,:,0]
        HB, _ , _ , _ =Histograma(imagen_hsv[:,:,0])
        HG, _ , _ , _ =Histograma(imagen_hsv[:,:,1]) 
        HR, _ , _ , _ =Histograma(imagen_hsv[:,:,2])  
        #arrayh = componente_h.flatten()
        #pasar imagen hsv en histograma 

        #male 1 - Female 0
        modname = NameDataset[i].replace(".jpg","")
        pos_agesex = saveas(modname)
        if pos_agesex <= len(Sex):
          Age_save = Age[pos_agesex]
          Sex_save = Sex[pos_agesex]

        total +=1

        HistB.append(HB)
        HistG.append(HG)
        HistR.append(HR)
        MagDat.append(Mag)
        FourDat.append(FourMag)

    return MagDat, FourDat, HistB, HistG, HistR

Descriptores Malignas

In [ ]:
MAGMal, FOURMal, HBMal, HGMal, HRMal = Descriptors(RutMalignas, NameMalig)
PromBM = np.mean(HBMal)
PromGM = np.mean(HGMal)
PromRM = np.mean(HRMal)


In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
#save
print(len(MAGMal))
with open('Malignas.json', 'w') as file:
    json.dump(MAGMal, file, indent=len(MAGMal), cls=NumpyEncoder)

2284


Descriptores Benignas
Pickle pithon -Buscar


In [ ]:
MAGBen, FOURBen, HBBen, HGBen, HRBen = Descriptors(RutBenignas, NameBenig)
PromBB = np.mean(HBBen)
PromGB = np.mean(HGBen)
PromRB = np.mean(HRBen)


In [ ]:
print(len(MAGBen))
with open('Benignas.json', 'w') as file:
    json.dump(MAGBen, file, indent=len(MAGBen), cls=NumpyEncoder)

7131


In [ ]:
NameBen = '/content/drive/My Drive/DataMB/DataBen.pkl'
NameMal = '/content/drive/My Drive/DataMB/DataMal.pkl'
DataBen = pd.DataFrame(MAGBen)
DataMal = pd.DataFrame(MAGMal)

DataBen.to_pickle(NameBen)
DataMal.to_pickle(NameMal)

In [ ]:
print(DataBen)

             0           1           2    ...        357        358        359
0      27.076906   24.638656   22.200784  ...  25.824285  25.824285  25.824285
1      21.631767   21.631767   20.638910  ...  21.638809  21.638809  21.638809
2      81.087037   83.885525   82.383949  ...  80.794561  80.794561  80.794561
3     225.741093  225.741093  226.341626  ...  -3.895572  -3.895572  -3.895572
4      49.802476   49.024992   46.990678  ...  47.772228  49.177362  49.177362
...          ...         ...         ...  ...        ...        ...        ...
7126  122.240824   27.046270   20.472265  ...  79.848429  79.848429  79.848429
7127   96.584935   93.643605   91.409097  ...  55.743537  56.648306  53.485886
7128  175.615325  166.631779  158.132881  ...  58.066002  40.031316  25.197008
7129   21.402532   15.382452    7.415370  ...  21.056960  21.056960  21.056960
7130  245.379292  245.379292  244.526146  ...  13.992358  13.992358  13.992358

[7131 rows x 360 columns]


In [ ]:
!cp -r /content/Malignas.json '/content/drive/My Drive/DataMB'

In [ ]:
!cp -r /content/Benignas.json '/content/drive/My Drive/DataMB'

cp: error writing '/content/drive/My Drive/DataMB/Benignas.json': No space left on device


Distancia Euclidea

In [ ]:
def euclidea(PromB, PromG, PromR, HB, HG, HR):
    ECl=[]
    for i in range(len(HB)):
        ECB = np.sqrt((np.power((PromB-HB[i]),2))+(np.power((PromG-HG[i]),2))+(np.power((PromR-HR[i]),2)))
        ECl.append(ECB)

    return ECl

In [ ]:
EUB = euclidea(PromBB, PromGB, PromRB, HBBen, HGBen, HRBen)

EUM = euclidea(PromBM, PromGM, PromRM, HBMal, HGMal, HRMal)

print(EUB)
print(EUM)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Pruebas 

In [ ]:
RutMalignas_p = '/usr/local/dataset/malignas_P'
RutBenignas_p = '/usr/local/dataset/benignas_P'
NameMalig_p = Ruta(RutMalignas_p)
NameBenig_p = Ruta(RutBenignas_p)

In [ ]:
#malignas
MAGMalP, FOURMalP, HBMalP, HGMalP, HRMalP = Descriptors(RutMalignas_p, NameMalig_p)

HBMS = []
HGMS = []
HRMS = []
for i in range(12):
    HBM = HBMalP[i]-EUM
    HGM = HGMalP[i]-EUM
    HRM = HRMalP[i]-EUM

    HBMS.append(HBM)
    HGMS.append(HGM)
    HRMS.append(HRM)

print(HBMS[0])
print(HGMS[0])
print(HRMS[0])

[[[ -1078.5463 ]
  [-13083.505  ]
  [-11584.301  ]
  ...
  [ -1078.5463 ]
  [ -1078.5463 ]
  [ -1078.5463 ]]

 [[ -1078.5463 ]
  [  -732.1238 ]
  [  -349.1042 ]
  ...
  [ -1029.7706 ]
  [ -1078.5463 ]
  [ -1011.846  ]]

 [[ -1078.5463 ]
  [  -773.52124]
  [  -396.38416]
  ...
  [ -1078.5463 ]
  [ -1078.5463 ]
  [ -1078.5463 ]]

 ...

 [[ -1078.5463 ]
  [ -8724.958  ]
  [-11043.812  ]
  ...
  [ -1056.3032 ]
  [ -1078.5463 ]
  [ -1071.6003 ]]

 [[ -1078.5463 ]
  [ -6235.9497 ]
  [ -8029.706  ]
  ...
  [ -1075.6414 ]
  [ -1078.5463 ]
  [ -1077.9641 ]]

 [[ -1078.5463 ]
  [ -5388.185  ]
  [ -6626.635  ]
  ...
  [ -1078.5463 ]
  [ -1078.5463 ]
  [ -1078.5463 ]]]
[[[ -1078.5463]
  [-13383.505 ]
  [-12238.301 ]
  ...
  [ -1078.5463]
  [ -1078.5463]
  [ -1078.5463]]

 [[ -1078.5463]
  [ -1032.1238]
  [ -1003.1042]
  ...
  [ -1029.7706]
  [ -1078.5463]
  [ -1011.846 ]]

 [[ -1078.5463]
  [ -1073.5212]
  [ -1050.3842]
  ...
  [ -1078.5463]
  [ -1078.5463]
  [ -1078.5463]]

 ...

 [[ -1078.5463]


In [ ]:
#benignas
MAGBenP, FOURBenP, HBBenP, HGBenP, HRBenP = Descriptors(RutBenignas_p, NameBenig_p)

HBBS = []
HGBS = []
HRBS = []
for i in range(12):
    HBB = HBBenP[i]-EUB
    HGB = HGBenP[i]-EUB
    HRB = HRBenP[i]-EUB

    HBBS.append(HBB)
    HGBS.append(HGB)
    HRBS.append(HRB)

print(HBBS)
print(HGBS)
print(HRMS)

[array([[[ -1414.3933],
        [ -1414.3933],
        [ -1414.3933],
        ...,
        [ -1414.3933],
        [ -1414.3933],
        [ -1414.3933]],

       [[ -1414.3933],
        [-66998.336 ],
        [-60411.43  ],
        ...,
        [ -1414.3933],
        [ -1414.3933],
        [ -1414.3933]],

       [[ -1414.3933],
        [ -1392.8171],
        [ -1900.2694],
        ...,
        [ -2785.203 ],
        [ -1405.7307],
        [ -4680.9365]],

       ...,

       [[ -1414.3933],
        [ -1292.148 ],
        [ -2914.769 ],
        ...,
        [-26072.607 ],
        [ -1414.3933],
        [-63177.67  ]],

       [[ -1414.3933],
        [ -1171.1744],
        [ -1277.8876],
        ...,
        [ -1414.3933],
        [ -1414.3933],
        [ -1413.8126]],

       [[ -1414.3933],
        [ -3073.5488],
        [ -6137.3384],
        ...,
        [ -1414.3933],
        [ -1414.3933],
        [ -1414.3933]]], dtype=float32), array([[[ -1414.3933],
        [ 19683.607 ],
      